In [1]:
import numpy as np




MC = np.array([ [0,1,2],
                [0,2,3],
              ])


MN = np.array([ [0,0,0],
                [0,10,0],
                [20,10,0],
                [20,0,0],
              ])


MNE = np.array([ [0,0,0],
                 [1,0,0] ])
MNT = np.array([ [2,5000,0],
                 [3,5000,0] ])


##########################################################

# MC = np.array([ [2,3,4],
#                 [0,1,4],
#                 [3,0,4],
#                 [1,2,4],
#               ])


# MN = np.array([ [0,0],
#                 [0,10],
#                 [20,10],
#                 [20,0],
#                 [10,5],
#               ])



# MNE = np.array([ [0,0,0],
#                  [1,0,0] ])
# MNT = np.array([ [4,0,0],
#                  [2,5000,0],
#                  [3,5000,0] ])
#####################################################################

espesor = 1
E = 30E6
v = 0.3


In [2]:
D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []
Areas = []

In [3]:
for i in range (len(MC)):
    
    i,m,j = MC[i]    

    x = np.array([MN[i,0],MN[m,0],MN[j,0]])
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])
    Bi = y[2] - y[1] ;   Bm = y[0] - y[2] ;    Bj = y[1] - y[0]
    Gi = x[1] - x[2] ;   Gm = x[2] - x[0] ;    Gj = x[0] - x[1]  
    
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    Area = np.linalg.det(Matriz_coordenadas)/2
    
    Areas.append(Area)

    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bj,0,Bm,0],[0,Gi,0,Gj,0,Gm],[Gi,Bi,Gj,Bj,Gm,Bm]])
    
    
    Matrices_Bloc.append(Matriz1)
    
    Matriz_local = espesor * abs(Area) * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,j*2,j*2+1,m*2,m*2+1]) #aca pasa algo raro
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    
print(Matriz_global*(0.91/375000))

[[ 48.   0. -28.  14.   0. -26. -20.  12.]
 [  0.  87.  12. -80. -26.   0.  14.  -7.]
 [-28.  12.  48. -26. -20.  14.   0.   0.]
 [ 14. -80. -26.  87.  12.  -7.   0.   0.]
 [  0. -26. -20.  12.  48.   0. -28.  14.]
 [-26.   0.  14.  -7.   0.  87.  12. -80.]
 [-20.  14.   0.   0. -28.  12.  48. -26.]
 [ 12.  -7.   0.   0.  14. -80. -26.  87.]]


In [5]:
Matrices_Bloc

[array([[-0.  , -0.  , -0.05, -0.  ,  0.05, -0.  ],
        [-0.  ,  0.1 , -0.  , -0.  , -0.  , -0.1 ],
        [ 0.1 , -0.  , -0.  , -0.05, -0.1 ,  0.05]]),
 array([[ 0.05, -0.  , -0.05, -0.  , -0.  , -0.  ],
        [-0.  , -0.  , -0.  ,  0.1 , -0.  , -0.1 ],
        [-0.  ,  0.05,  0.1 , -0.05, -0.1 , -0.  ]])]

In [4]:
for i in Matrices_Bloc:
    print(i*2*Area)

[[  0.   0.  10.   0. -10.   0.]
 [  0. -20.   0.   0.   0.  20.]
 [-20.   0.   0.  10.  20. -10.]]
[[-10.   0.  10.   0.   0.   0.]
 [  0.   0.   0. -20.   0.  20.]
 [  0. -10. -20.  10.  20.   0.]]


In [6]:
Desplazamientos_f = np.zeros(len(MN)*2)
Fuerza = []
Nodos = []


for i in MNE:
    nodo,valorx,valory = i 
    Desplazamientos_f[2*nodo] = valorx
    Desplazamientos_f[2*nodo+1] = valory


for i in MNT:
    nodo,valorx,valory = i
    Fuerza.append(valorx)
    Fuerza.append(valory)
    Nodos.append(2*nodo)
    Nodos.append(2*nodo+1) 

print(Fuerza)
print(Nodos)

Fuerza = np.array(Fuerza).reshape(-1,1)

Matriz_res = Matriz_global[np.ix_(Nodos,Nodos)]

Desplazamientos_parcial = np.linalg.solve(Matriz_res,Fuerza)


o = 0
for i in Nodos:
    Desplazamientos_f[i] = Desplazamientos_parcial [o]
    o += 1
  

[5000, 0, 5000, 0]
[4, 5, 6, 7]


In [9]:
Desplazamientos_f

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       6.09580998e-04, 4.16333066e-06, 6.63704297e-04, 1.04083267e-04])

In [7]:
Fuerzas_ = np.dot(Matriz_global,Desplazamientos_f)
print(Fuerzas_.round(4))
print()
print(Desplazamientos_f*1e6)

[-5000.     -3002.4019 -5000.      3002.4019  5000.         0.
  5000.         0.    ]

[  0.           0.           0.           0.         609.58099813
   4.16333066 663.70429677 104.08326661]


In [11]:
#Esfuerzos

Tensiones = []

for p in range(len(MC)):
    
    i,m,j = MC[p]   #MDF_COMMENT si definis un orden de nodos aca, tenes que usar el mismo para los desplazamientos.
    
    print(i,j,m)
    
#MDF_COMMENT    nodos = [2*i,2*i+1,2*j,2*j+1,2*m,2*m+1]   <<<<< me parecia que no entendia algo. 
    nodos = [2*i,2*i+1,2*m,2*m+1,2*j,2*j+1]
    
    print(Desplazamientos_f[nodos])
    # me parece que no es Desplazamientos_f sino 
    A = np.dot(Matrices_Bloc[p],Desplazamientos_f[nodos])
    
    print(Matrices_Bloc[p]*2*Areas[p])
    
    print(Areas[p])
    
    B = np.dot(D,A)
    
   
    Tensiones.append(B)
    
for i in range (len(Tensiones)):
    print(Tensiones[i])
    print()

0 2 1
[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 6.09580998e-04 4.16333066e-06]
[[  0.   0.  10.   0. -10.   0.]
 [  0. -20.   0.   0.   0.  20.]
 [-20.   0.   0.  10.  20. -10.]]
-100.00000000000006
0 3 2
[0.00000000e+00 0.00000000e+00 6.09580998e-04 4.16333066e-06
 6.63704297e-04 1.04083267e-04]
[[-10.   0.  10.   0.   0.   0.]
 [  0.   0.   0. -20.   0.  20.]
 [  0. -10. -20.  10.  20.   0.]]
-100.00000000000006
[1000.6862633   287.715887   -700.96076861]

[-1103.62575775  -630.84753517   -64.85188151]

